In [1]:

import os
import time
import oss2
import json

import cv2 as cv
import numpy as np
from tqdm import tqdm
from sko.GA import GA

from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkfacebody.request.v20191230.CompareFaceRequest import CompareFaceRequest


In [2]:
import oss2
import aliyunsdkcore
import aliyunsdkfacebody
aliyunsdkcore.__version__, aliyunsdkfacebody.__version__, oss2.__version__


('2.13.11', '1.2.0', '2.13.0')

In [3]:

access_key_id = os.getenv('OSS_TEST_ACCESS_KEY_ID', '')
access_key_secret = os.getenv('OSS_TEST_ACCESS_KEY_SECRET', '')
bucket_name = os.getenv('OSS_TEST_BUCKET', 'ivan-bucket-out-002')
endpoint = os.getenv('OSS_TEST_ENDPOINT', 'oss-cn-shanghai.aliyuncs.com')

bucket = oss2.Bucket(oss2.Auth(access_key_id, access_key_secret), endpoint, bucket_name)

client = AcsClient(access_key_id, access_key_secret, 'cn-shanghai')


In [4]:

#
def _f1(imgA=None, imgB=None, Rect=None):
    while True:
        try:
            time.sleep(0.68)
            if imgA:
                oss2.resumable_upload(bucket, 'imgA.jpg', imgA)
            if imgB:
                oss2.resumable_upload(bucket, 'imgB.jpg', imgB)

            request = CompareFaceRequest()
            request.set_accept_format('json')

            request.set_ImageURLA(f"https://{bucket_name}.{endpoint}/imgA.jpg")
            request.set_ImageURLB(f"https://{bucket_name}.{endpoint}/imgB.jpg")

            response = client.do_action_with_exception(request)
            
            if Rect:
                return json.loads(response)["Data"]["Confidence"], json.loads(response)["Data"]["RectAList"]
            else:
                return json.loads(response)["Data"]["Confidence"]
        except Exception as e:
            print(e)

#
def _f2(imgA, imgB):
    imgA, imgB = cv.imread(imgA), cv.imread(imgB)
    return np.sqrt(np.sum(np.square(imgA - imgB)))


In [5]:
%time _f1(imgA="/Users/ivan/Desktop/企业微信截图_16195386449700.png",imgB="/Users/ivan/Desktop/企业微信截图_1619538664196.png")

CPU times: user 20.4 ms, sys: 6.56 ms, total: 27 ms
Wall time: 2.3 s


70.34921264648438